In [ ]:
!pip install transformers
!pip install pandas requests beautifulsoup4 openpyxl
!pip install gtts openpyxl


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from transformers import pipeline
from gtts import gTTS
import os

def Scraping_folha(limit=10):
    site_Folha = requests.get('https://www1.folha.uol.com.br/ultimas-noticias/')
    bs = BeautifulSoup(site_Folha.content, 'html.parser')

    noticias = bs.find_all('div', 'c-headline__content')
    ultimas_noticias = []
    for n in noticias:
        info = 'Folha de S.Paulo (Brasil, São Paulo, São Paulo) - Português'
        Link = n.find('a')['href']
        Manchete = n.find('h2').text
        Data = n.find('time')['datetime']
        ultimas_noticias.append({'Manchete': Manchete, 'Link': Link, 'Data': Data, 'Informações': info})

    ultimas_folha = pd.DataFrame(ultimas_noticias[:limit])
    return ultimas_folha

def analyze_sentiment(df):
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    results = sentiment_analysis(df['Manchete'].tolist())

    df['Sentiment'] = [result['label'] for result in results]
    df['Score'] = [result['score'] for result in results]
    df['Classificação'] = df['Sentiment'].apply(lambda x: 'Positiva' if int(x.split()[0]) > 3 else 'Negativa')

    return df

def generate_audio(df):
    text = ""
    for index, row in df.iterrows():
        text += f"Manchete {index + 1}: {row['Manchete']}. "

    tts = gTTS(text, lang='pt')
    audio_file = 'manchetes.mp3'
    tts.save(audio_file)
    print(f"Arquivo de áudio gerado: {audio_file}")

def main():
    ultimas_folha = Scraping_folha(limit=10)
    ultimas_folha_with_sentiment = analyze_sentiment(ultimas_folha)

    # Salvar o DataFrame em um arquivo Excel
    excel_file = 'ultimas_folha_with_sentiment.xlsx'
    ultimas_folha_with_sentiment.to_excel(excel_file, index=False)
    print(f"Arquivo Excel gerado: {excel_file}")

    # Gerar o arquivo de áudio
    generate_audio(ultimas_folha_with_sentiment)

    print(ultimas_folha_with_sentiment)

    return ultimas_folha_with_sentiment

if __name__ == "__main__":
    df = main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Arquivo Excel gerado: ultimas_folha_with_sentiment.xlsx
Arquivo de áudio gerado: manchetes.mp3
                                            Manchete  \
0        Boulos e Nunes lideram com números modestos   
1                                 Eleições de araque   
2  EUA têm déficit agrícola recorde, e Brasil é u...   
3  Pedro Bial conta qual foi a sua edição preferi...   
4  Portugal pode contribuir para melhorar imagem ...   
5  Meu plano de saúde vai ser 'descancelado'? O q...   
6                  No Congresso, a derrota é do país   
7  'Ônibus marinho' começa a operar em Ilhabela a...   
8  Museu da Diversidade Sexual é reinaugurado nes...   
9  Grêmio volta ao futebol com vitória após pausa...   

                                                Link              Data  \
0  https://www1.folha.uol.com.br/opiniao/2024/05/...  2024-05-29 22:00   
1  https://www1.folha.uol.com.br/opiniao/2024/05/...  2024-05-29 22:00   
2  https://www1.folha.uol.com.br/colunas/vaivem/2...  2024-05-29 2